# Notebook 2: OpenFace Real-Time Gaze Estimation
This notebook captures RGB frames from the RealSense camera, feeds them to OpenFace, and visualizes the gaze vector in real time.
Make sure OpenFace is correctly installed and its binary is accessible from the command line.
Press 'q' to quit the live preview.

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import subprocess
import pandas as pd


In [ ]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)


In [ ]:
def run_openface_on_frame(image_path, output_dir="openface_output"):
    os.makedirs(output_dir, exist_ok=True)
    command = [
        "./OpenFace/build/bin/FeatureExtraction",  # path 
        "-f", image_path,
        "-out_dir", output_dir,
        "-gaze"
    ]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    gaze_data_path = os.path.join(output_dir, "gaze.csv")
    if os.path.exists(gaze_data_path):
        df = pd.read_csv(gaze_data_path)
        if len(df) > 0:
            return df.iloc[-1]
    return None


In [ ]:
print("Press 'q' to quit.")
frame_count = 0

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())
        frame_path = f"temp_frame_{frame_count}.png"
        cv2.imwrite(frame_path, color_image)

        # Run OpenFace
        gaze_data = run_openface_on_frame(frame_path)

        # Visualize
        if gaze_data is not None:
            eye_x = gaze_data['eye_lmk_x_0']
            eye_y = gaze_data['eye_lmk_y_0']
            gaze_x = gaze_data['gaze_0_x']
            gaze_y = gaze_data['gaze_0_y']

            pt1 = (int(eye_x), int(eye_y))
            pt2 = (int(eye_x + gaze_x * 100), int(eye_y + gaze_y * 100))
            cv2.arrowedLine(color_image, pt1, pt2, (0, 255, 0), 2)

        cv2.imshow("OpenFace Gaze Estimation", color_image)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

        frame_count += 1

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
